* *tecniche di compressione dei modelli*
    * servono a ridurre la dimensione, la complessità computazionale ed il consumo energetico di una rete neurale
        * si cerca di mantenere le stesse prestazioni della rete di riferimento (quella non compressa)
        * utilizzo più efficiente della memoria e dell'energia e riduzione dei tempi di computazione (training ed inferenza)
        * il modello compresso può essere eseguito su dispositivi con risorse limitate
            * smartphone, dispositivi edge ...
    * l' autore individua sei tecniche di compressione
        * *Down-sizing dei modelli*
            * si creano modelli densi più piccoli per risolvere lo stesso problema
            * tecniche di esempio
                * distillazione del modello
                * Neural Architecture Search
                * consentono di trovare modelli densi più piccoli    
        *  *Fattorizzazione degli operatori*
            * porta ad una decomposizione degli operatori
            * le matrici associate a strati densi vengono decomposte in matrici con rango ridotto
            * matrici più piccole
                * comportano meno operazioni
                * comportano meno occupazione di memoria  
        *  *Quantizzazione dei valori:*
            * si riduce la precisione numerica dei valori nella rete (come i pesi, attivazioni o gradienti)
            * i valori vengono codificati con sistemi che utilizzano un minor numero di bit
            * diversi modi di codificare interi e numeri in virgola mobile che consentono un risparmio in termini di bit rispetto ai tipi di dati standard che prevedono 32 o 64 bit 
        *  *Compressione dei valori*
            * consentono la compressione della struttura e dei valori (es. pesi)
                * attraverso metodi basati sull'entropia o sulla perdita
                * utilizzano la correlazione fra i valori 
        *  *Condivisione dei parametri*
            * sfruttano la ridondanza dei parametri per comprimere un modello
            * la ridondanza può essere agevolata durante il processo di training attraverso opportuni metodi
            * (condivisione fa pensare al fatto che diversi parametri abbiano lo stesso valore e quindi possono essere condivisi in qualche modo)?
        *  *Sparsificazione*
            * riduce la complessità di un modello azzerando molti dei suoi parametri
    * i metodi sopra elencati portano ad una riduzione dei requisiti spaziali (di memoria) e computazionali
        * ad eccezione del metodo basato sulla condivisione che non è detto che comporti un miglioramento dei costi computazionali
            * sebbene contribuisca comunque alla riduzione dimensionale del modello
    * l'applicazione dei metodi non è esclusiva
        * è possibile combinare insieme più metodi per ottimizzare la compressione (esistono dei lavori in letteratura al riguardo)
* l'articolo focalizza sul metodo di sparsificazione noto anche come "pruning"
    * metodo ritenuto molto efficace relativamente al task della compressione di un modello
  

I *modelli di deep learning*, anche noti come *reti di deep learning* o *reti neurali artificiali* possono essere visti come grafi composti da strati parametrizzabili, detti operatori, che insieme implementano una funzione complessa di tipo non lineare $f$. Consideriamo un apprendimento di tipo supervisionato dove abbiamo a disposizione un training set (esempi "etichettati") composto da un insieme di coppie $(\mathbf{x, y})$ per $\mathbf{x} \in \chi$ ed $\mathbf{y} \in \mathcal{Y}$

+ l'obiettivo dell'apprendimento è quello di appunto apprendere una funzione $f \colon \chi \to \mathcal{Y}$
    + la funzione $f$ è parametrizzata dai pesi $\mathbf{w} \in \mathbb{R}^d$
    + l'apprendimento consiste nel trovare un opportuno valore per il vettore dei parametri $\mathbf{w}$
        + in modo che dati un valore di input $\mathbf{x} \in \chi$ ed un valore di output $\mathbf{y} \in \mathcal{Y}$ deve aversi che $f(\mathbf{x;w}) \approx \mathbf{y}$
+ l'applicazione della funzione $f$ al vettore di input $\mathbf{x}$ può essere interpretata come la trasformazione di $\mathbf{x}$ strato per strato fino alla generazione dell'output $\mathbf{y}$
    + ogni strato trasoforma l'input in una nuova rappresentazione la quale poi viene presentata come input per lo strato successivo e così via
    + per una rete addestrata questo processo di trasformazione prende il nome di *inferenza*
    + durante la fase di training questo processo prende il nome di *forward pass*
+ dato un problema possiamo provare a vedere se esista una rete neurale idonea alla sua soluzione
    + questo processo di ricerca di una rete che sia capace di risolvere un dato problema può essere decomposto in due fasi
        + *design*
            + progettare una opportuna architettura per la rete
                + dare alla rete una struttura appropriata alla soluzione del problema dato 
        + *training*
            + dopo la fase di progettazione si esegue l'addestramento della rete
            + obiettivo della fase di training è quello di trovare un valore ottimale per il vettore $\mathbf{w}$ dei pesi
+ l'architettura di una rete viene di solito progettata manualmente da un qualche architetto o gruppo di architetti
    + la struttura risultante non viene poi modificata durante la fase di addestramento
+ l'addestramento di una rete può essere un processo di tipo iterativo e quindi può essere ripetuto più e più volte
    + per ricercare una, per così dire, "ottimizzazione ottimale"
    + la qualità della trasformazione attuata dalla funzione $f(\mathbf{x;w})$ (forward pass) viene misurata attraverso una loss function $\mathscr{l} \colon \mathcal{Y} \times \mathcal{Y} \to \mathbb{R}$
        + la loss function $\mathscr{l}(\mathbf{y}, f(\mathbf{x;w}))$ fornisce una stima dell'accuratezza della precisione
        + funzioni tipiche usate come loss function sono la norma $L_2$ oppure la cross-entropy
    + al forward pass fa seguito la fase di backward pass
        + in questa fase la perdita o errore si propaga all'indietro nella rete cioè dallo strato di output verso lo strato di input
        + in corrispondenza di ogni strato il backward pass calcola il "gradiente" $g_k = \partial \ell / \partial g_k$ ed applica una regola di appredimento (learning rule) per aggiornare i pesi $w_k$ in modo che la loss function $\ell$ diminuisca
            + questo viene fatto per ogni coppia $(\mathbf{x, y})$ presente nell'insieme di training
    + l'accuratezza ottenuta durante la fase di apprendimento viene misurata utilizzando un insieme di valori di $\mathbf{x}$ mai visti prima
        + tale insieme prende il nome di *test set*
        + questa fase avrebbe lo scopo di valutare la capacità di generalizzazione del modello
            + fare previsioni sull'output, a partire da input mai visti, che siano "abbastanza" vicine all'output reale
            + esiste sempre il rischio di overfitting o sovra-adattamento
                + il modello diventa bravissimo a prevedere solo ciò che ha già visto durante l'addestramento ma non è in grado di produrre previsioni corrette
                    + cioè la probabilità che l'output sia "molto errato" (la distanza dal valore reale è grande) è elevata
+ Consideriamo il caso del *percettrone multistrato* (Multilayer Perceptron o MLP)
    + è un modelli di rete neurale artificiale composta da più strati (multilayer)
    + ogni strato è connesso al successivo in accordo alla configurazione completamente connessa (fully connected)
        + ogni neurone di uno strato è connesso a tutti i neuroni dello strato successivo
            + più precisamente l'output di ogni neurone di uno strato viene portato in ingresso ad ogni neurone dello strato successivo 
            + escluso lo strato di uscita che non possiede uno strato successivo
            + ma cosa è un neurone?
                + è l'unità di base di una rete neurale artificiale la quale è composta da tanti neuroni connessi fra loro secondo qualche configurazione
                + è un modello matematico molto semplificato del corrispondente neurone biologico
                + .....
                + **continua il discorso sul modello di McCulloch e Pitts**  
    + ![Multilayer Perceptron structure](\img\MLP70.png)
    + il modello ha uno strato di input $\mathbf{x}_0$, due strati nascosti (hidden layer) $\mathbf{x_1, x_2}$ ed uno strato di uscita $\mathbf{x}_3$ con funzione di attivazione ReLU (rectified linear unit) definita come $\sigma_R(x) = \max(0, x)$
    + il numero di neuroni presenti all'interno dello strato $i$ viene denotato con il simbolo $\lvert x_i \rvert$
    + la fase forward pass può essere descritta nel seguente modo
        + $f(\mathbf{x_0;w}) = \sigma_R(\mathbf{w_3} \cdot \sigma_R(\mathbf{w_2} \cdot \sigma_R(\mathbf{w_1x_0 + b_1) + b_2) + b_3)}$ dove $\mathbf{x_0}$ è il vettore di input, $\mathbf{w_k}$ sono le matrici dei pesi associati allo strato $k$ mentre $\mathbf{b_k}$ sono i bias asociati allo strato $k$-esimo
        + ![same but spasified MLP network](\img\sparsifiedMLP60.png)
        + sopra la stessa rete neurale ma sparsificata
        + azzerando una riga di una matrice $\mathbf{w}_k$ si elimina un neurone dallo strato $k$
        + azzerando i valori all'interno di una colonna di una matrice $\mathbf{w}_k$ si eliminano delle connessioni
            + cioè delle caratteristiche di input
+ l'addestramento di una rete neurale profonda può essere visto come il processo di minimizzazione di una data loss function utilizzando i dati presenti nel training set
    + cioè si tratta di attuare un processo che porta a determinare quei valori dei pesi che rendono minima la funzione di perdita relativamente ai dati di addestramento
        + pericolo di overfitting e bassa capacità di generalizzazione da parte del modello
+ possiamo definire la funzione di perdita $L$ utilizzata durante il training come
    + $L(\mathbf{w}) = \frac{1}{N} \sum_{n=1}^N \ell(\mathbf{y}[n], f(\mathbf{x}[n];\mathbf{w})$
    + quindi la loss function sarà una funzione $L \colon \mathbb{R}^d \to \mathbb{R}$ (dove $\mathbf{w} \in \mathbb{R}^d$)
+ il metodo di addestramento più utilizzato è quello basato sul gradiente discente stocastico (Stochastic Gradient Descent o SGD)
    + (***si potrebbe fornire una breve descrizione di come funziona il metodo del gradiente discendente per ricavare una approssimazione del minimo di una data funzione***)
    + le derivate parziali $\partial L/\partial w_k$ della funzione $L$ rispetto ai pesi $w_k$ possono essere calcolate, per un dato esempio $\mathbf{x}$ usando un metodo di derivazione automatizzato
    + ***si potrebbe accennare al funzionamento della differenziazione automatizzata***
        + il metodo SGD richiede il calcolo del gradiente $\nabla L$
            + il gradiente ci dice qual'è la direzione di massima crescita di una funzione in un dato punto del suo dominio
                + e dunque dal punto considerato ci si deve spostare un pochino (che dipende dall'iper-parametro noto come learning rate) in direzione opposta a quella del gradiente che rappresenta la direzione di minima crescita
    + il gradiente calcolato viene poi utilizzato per applicare una regola di apprendimento (learning rule) $R$
        + la regola definisce come aggiornare i pesi prima della successiva iterazione: $\mathbf{w}^{(i+1)} = \mathbf{w}^{(i)} + R(\mathbf{g,w^{(i)}})$
            + il nuovo peso viene calcolato a partire da quello vecchio, aggiungendo qualcosa (di positivo o negativo) ad esso calcolato secondo una specifica regola $R$
        
+ parliamo ora della matrice Jacobiana
    + sia $F \colon \mathbb{R}^d \to \mathbb{R}^m$ una funzione di più variabili a valori vettoriali
    + si tratta di una matrice i cui elementi sono rappresentati dalle derivate parziali prime della funzione $F$
        + cioè ha per righe i gradienti delle componenti di $F$
        + se $m = 1$ la matrice si riduce ad un vettore riga e corrisponde al gradiente vero e proprio
        + possiamo perciò vedere la matrice Jacobiana come una generalizzazione del concetto di gradiente per funzioni a valori vettoriali
        + nel nostro caso, data $L$ come funzione di perdita, la sua matrice Jacobiana è data da
            + $\mathbf{J} = \nabla_\mathbf{w} L = \begin{bmatrix} \frac{\partial L}{\partial w_1} & \frac{\partial L}{\partial w_2} & \cdots & \frac{\partial L}{\partial w_d} \end{bmatrix} = \begin{bmatrix} g_1 & g_2 & \cdots & g_d \end{bmatrix}$

+ parliamo adesso della matrice Hessiana
    + la matrice Hessiana di una funzione di più variabili a valori in un campo scalare è una matrice quadrata $n \times n$ i cui elementi sono rappresentati dalle derivate parziali seconde della funzione
    + più formalmente, data la funzione $f \colon \mathbb{R}^n \to \mathbb{R}$, nell'ipotesi di esistenza di tutte le sue derivate parziali seconde, si definisce come matrice Hessiana della funzione $f$ la matrice $\mathbf{H_f}(\mathbf{x})$ data da
$$
    \mathbf{H_f} = 
    \begin{bmatrix}
        \frac{\partial^2 f}{\partial x_1^2} & \frac{\partial^2 f}{\partial x_1 \partial x_2 } & \cdots & \frac{\partial^2 f}{\partial x_1 \partial x_n} \\      \frac{\partial^2 f}{\partial x_2 \partial x_1} & \frac{\partial^2 f}{\partial x_2^2} & \cdots & \frac{\partial^2 f}{\partial x_2 \partial x_n} \\
        \vdots & \vdots & \ddots & \vdots \\
        \frac{\partial^2 f}{\partial x_n \partial x_1} & \frac{\partial^2 f}{\partial x_n \partial x_2 } & \cdots & \frac{\partial^2 f}{\partial x_n^2}
    \end{bmatrix}
$$
    + per una loss function $L$ differenziabile due volte la sua matrice Hessiana si può rappresentare come $\mathbf{H} = \nabla_\mathbf{w}^2 L$
    + la matrice Hessiana fornisce informazioni relative alla geometria locale ovvero alla curvatura di una funzione in un intorno di un punto $\mathbf{w}$
    + sfruttando il teorema di Taylor per funzioni di più variabili è possibile ottenere una espressione che approssima al secondo ordine una funzione in un intorno di raggio $\delta$ di un punto dato $\mathbf{w}$ utilizzando il gradiente e la matrice Hessiana
    + possiamo quindi scrivere
$$
    L(\mathbf{w} + \delta \mathbf{w}) \approx L(\mathbf{w}) + \nabla_\mathbf{w} L \delta \mathbf{w} + \frac{1}{2} \delta \mathbf{w}^T \mathbf{H} \delta \mathbf{w}.
$$
    + l'approssimazione sopra descritta prende anche il nome di modello quadratico locale per la loss function

+ potremmo pensare che ad ogni esempio $\mathbf{x}$ corrisponda una certa probabilità che la rete fornisca come risultato $\mathbf{y}$
+ siano dati un insieme di input $\mathbf{x} \in \chi$ e un insieme di corrispondenti output $\mathbf{y} \in \mathcal{Y}$
    + ipotizziamo che i vettori di input $\mathbf{x}$ siano estratti da una distribuzione $Q_\mathbf{x}$ e che i corrispondenti valori di output $\mathbf{y}$ siano presi da una distribuzione condizionale $Q_\mathbf{y|x}$
        + la distribuzione di probabilità congiunta può essere definita come $Q_\mathbf{x,y} = Q_\mathbf{x}Q_\mathbf{y|x}$
    + l'obiettivo è quello di minimizzare la distanza fra la probabilità congiunta target $Q_\mathbf{x,y}$ e quella appresa $P_\mathbf{x,y}(\mathbf{w})$
        + è comune usare come misura della distanza fra due distribuzioni di probabilitàla divergenza di Kullback-Leibler (indicata con KL)
            + misura la distanza (non simmetrica) fra due distribuzioni di probabilità $P$ e $Q$
            + si indica con il simbolo $D_{KL}(P\lVert Q)$ che significa divergenza di $Q$ da $P$ secondo Kullback-Leibler
    + addestrare il modello significa fare in modo che esso apprenda la distribuzione di probabilità condizionata $P_{y|x}(\mathbf{w})$ che sia più "vicina" possibile alla reale distribuzione $Q_{y|x}$
+ ***continua il discorso sulla prospettiva probabilistica dei modelli di ML***

### Panoramica sulla sparsificazione

Perché sparsificare un modello? Principalmente per due ragioni: 1) per migliorare le prestazioni relativamente all'addestramento e all'inferenza; 2) per migliorare la capacità del modello di generalizzare (e migliorare la sua robustezza). (Aggiungerei anche miglioramenti legati al consumo energetico poiché, in generale, ad un minor costo computazionale corrisponde un minore consumo di energia). Bisogna osservare che la sparsificazione richiede un compromesso fra risparmio in termini computazionali e spaziali e accuratezza del modello. Una estrema sparsificazione porta con certezza ad una riduzione significativa dell'accuratezza del modello. Questo perché il ridotto numero di parametri non consente al modello di apprendere con profondità dai dati di training.
La capacità di un modello di generalizzare rappresenta un aspetto molto importante nel deep learning e rappresenta una misura di quanto un modello riesca ad eseguire previsioni corrette su dati di input mai visti prima. ALcuni schemi di sparsificazione sembrano seguire la teoria del rasoio di Occam (la quale afferma che se per spiegare una fenomeno esistono diverse teorie ugualmente valide, è preferibile scegliere quella più semplice)

![Accuracy Vs. Sparsity](\img\accuracyVsSparsity.png)

Come si può osservare dalla figura sopra, all'aumentare della sparsità vi è, inizialmente un leggero aumento dell'accuratezza e delle prestazioni. Questo fenomeno può essere spiegato sulla base del fatto che in un modello più piccolo l'algoritmo di apprendimento focalizza maggiormente sugli aspetti più importanti e e generali (?). Aumentando ancora la sparsità, l'accuratezza, ad un certo punto raggiunge un valore massimo e le prestazioni migliorano ulterioremente. Ora aumentando ancora il livello di sparsità l'accuratezza si degrada molto velocemente mentre le prestazioni sembrano tendere asintoticamente ad un certo valore massimo. Da qui nasce la necessità di compromesso, come del resto avviene in ogni progetto ingegneristico.
Risulta complesso il problema del raggiungimento delle migliori prestazioni possibili dato un specifico livello di sparsità. Esistono delle tecniche di archiviazione e di computazione che consentono di sfruttare al meglio la sparsità ma solo all'interno di un determinato intervallo o solo quando gli elementi non nulli siano distribuiti in specifici modi.